In [ ]:
# ⏱ Core & Utilities
import time            # ⏳ Sleep, delays, timing
import json            # 📦 Parse JSON responses
import html            # 📝 Decode HTML entities
import string          # 🔤 Letters, digits, etc.
import pandas as pd    # 📊 Data handling

# 🌐 Selenium & Browser Automation
import undetected_chromedriver as uc                   # 🕵️‍♂️ Stealth Chrome driver
from selenium.webdriver.common.by import By           # 🔎 Locate elements
from selenium.webdriver.support.ui import WebDriverWait  # ⏳ Explicit waits
from selenium.webdriver.support import expected_conditions as EC  # ✅ Wait conditions

# 📈 Progress & Display
from tqdm import tqdm  # 🔄 Progress bars

In [ ]:
# 🔤 Configuration
CHARS = string.ascii_uppercase   # All letters A-Z for band search
PAGE_SIZE = 500                  # Max records per page

all_bands = []                   # 📝 To store all scraped bands

# 🌐 Initialize Chrome (undetected)
driver = uc.Chrome(headless=False)  # headless=False for visual debugging

try:
    # 🔁 Loop through each letter
    for char in CHARS:
        # 🛠 First request to get total number of bands for this letter
        first_url = f"https://www.metal-archives.com/browse/ajax-letter/l/{char}/json?iDisplayStart=0&iDisplayLength={PAGE_SIZE}"
        driver.get(first_url)
        time.sleep(1)  # ⏳ Wait for page/load
        
        pre_text = driver.find_element(By.TAG_NAME, "pre").text
        data = json.loads(pre_text)
        total_records = data["iTotalRecords"]

        if total_records == 0:
            print(f"No bands found for letter {char} ❌")
            continue

        pages = range(0, total_records, PAGE_SIZE)
        print(f"Scraping {total_records} bands for letter {char} ({len(pages)} pages) 🔍")

        # 🌀 Loop through pages for this letter
        for start in tqdm(pages, desc=f"Letter {char}", unit="page"):
            url = f"https://www.metal-archives.com/browse/ajax-letter/l/{char}/json?iDisplayStart={start}&iDisplayLength={PAGE_SIZE}"
            driver.get(url)
            time.sleep(1)  # polite delay

            # 📦 Parse JSON response
            pre_text = driver.find_element(By.TAG_NAME, "pre").text
            page_data = json.loads(pre_text)

            # 🧩 Extract band details
            for band in page_data["aaData"]:
                name_html, country, genre, status_html = band

                # Extract band name from HTML
                name = html.unescape(name_html).split(">")[1].split("<")[0]
                # Extract band ID from URL
                url_band = html.unescape(name_html).split("'")[1]
                band_id = url_band.rstrip("/").split("/")[-1]
                # Extract status (active/unknown/etc.)
                status = html.unescape(status_html).split(">")[1].split("<")[0]

                # Append to master list
                all_bands.append({
                    "name": name,
                    "band_id": band_id,
                    "country": country,
                    "genre": genre,
                    "status": status
                })

finally:
    driver.quit()  # 🚪 Close browser safely

# 📊 Create final DataFrame and save to CSV
df = pd.DataFrame(all_bands)
print(f"Scraping complete! Total bands collected: {len(df)} ✅")
df.to_csv('metal_bands.csv', index=False)

In [ ]:
# 📝 Read the CSV into a DataFrame
df = pd.read_csv('metal_bands.csv')

# 👀 Preview the first few rows
df.head()

In [ ]:
# 🗂 Use the bands DataFrame from the first cell
bands_df = df  # Must contain 'name' and 'band_id'

# 📝 Container for all album data
all_albums = []

# 🚀 Initialize Chrome (headless=False for stability)
driver = uc.Chrome(headless=False)

total_bands = len(bands_df)
total_albums_collected = 0

try:
    # 🔄 Iterate through all bands with a single progress bar
    with tqdm(total=total_bands, desc="Bands scraped", unit="band") as pbar:
        for _, row in bands_df.iterrows():
            band_name = row["name"]
            band_id = row["band_id"]
            url = f"https://www.metal-archives.com/band/discography/id/{band_id}/tab/all"

            try:
                driver.get(url)

                # ⏳ Wait up to 5 seconds for the discography table to appear
                WebDriverWait(driver, 5).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "table.display.discog tbody tr"))
                )
            except:
                # ⚠ Skip bands with no discography or timeout
                pbar.set_postfix_str(
                    f"{band_name} ({band_id}): No discography/timeout | Total albums: {total_albums_collected}"
                )
                pbar.update(1)
                continue

            # 📑 Extract rows from the discography table
            rows = driver.find_elements(By.CSS_SELECTOR, "table.display.discog tbody tr")
            band_album_count = 0

            for tr in rows:
                cells = tr.find_elements(By.TAG_NAME, "td")
                if len(cells) < 4:
                    continue  # Skip malformed rows

                album_type = cells[1].text.strip()
                if album_type.lower() != "full-length":
                    continue  # Only full-length albums

                title = cells[0].text.strip()
                year = cells[2].text.strip()
                reviews_text = cells[3].text.strip()
                reviews = int(reviews_text) if reviews_text.isdigit() else 0

                all_albums.append({
                    "band_name": band_name,
                    "band_id": band_id,
                    "title": title,
                    "year": year,
                    "reviews": reviews
                })
                band_album_count += 1
                total_albums_collected += 1

            # 🔄 Update progress bar with band name, albums collected, and total albums
            pbar.set_postfix_str(
                f"{band_name} ({band_album_count} albums) | Total albums: {total_albums_collected}"
            )
            pbar.update(1)

            # ⏳ Polite delay to avoid overloading server
            time.sleep(0.5)

finally:
    driver.quit()

# 🗃 Create final DataFrame and preview
albums_df = pd.DataFrame(all_albums)
print(f"🎉 Scraped a total of {total_albums_collected} full-length albums from {total_bands} bands.")
albums_df.head()